In [500]:
import pandas as pd
import numpy as np

In [501]:
df = pd.read_csv('clean_formspring.csv')
df.head()

,Unnamed: 0,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3,overall_ans,overall_severity
0,0,aguitarplayer94,Q: what&#039;s your favorite song? :D<br>A: I ...,what&#039;s your favorite song? :D<br>,I like too many songs to have a favorite,None,0,0,NaN,0,0,NaN,0,0,NaN,0,0.0
1,1,aprilpooh15,Q: <3<br>A: </3 ? haha jk! <33,<3,</3 ? haha jk! <33,None,0,0,NaN,0,0,NaN,0,0,NaN,0,0.0
2,2,aprilpooh15,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,&quot;hey angel you duh sexy&quot;,Really?!?! Thanks?! haha,None,0,0,NaN,0,0,NaN,0,0,NaN,0,0.0
3,3,aprilpooh15,Q: (:<br>A: ;(,(:,;(,None,0,0,NaN,0,0,NaN,0,0,NaN,0,0.0
4,4,aprilpooh15,Q: ******************MEOWWW*******************...,******************MEOWWW*************************,*RAWR*?,None,0,0,NaN,0,0,NaN,0,0,NaN,0,0.0


In [502]:
df_new = df[['post', 'overall_ans', 'overall_severity']]
df_new.head()

,post,overall_ans,overall_severity
0,Q: what&#039;s your favorite song? :D<br>A: I ...,0,0.0
1,Q: <3<br>A: </3 ? haha jk! <33,0,0.0
2,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,0,0.0
3,Q: (:<br>A: ;(,0,0.0
4,Q: ******************MEOWWW*******************...,0,0.0


In [503]:
# removing noise from data
import re

def regex(post):
    post = re.sub(r'[a-zA-Z0-9._%+-] + @ + [a-zA-Z0-9.-] + (\.[a-zA-Z]{2,4})', '', post) # removing email
    post = re.sub(r'Q\:\s', '', post) # removing ques & ans header
    post = re.sub(r'A\:\s', '', post) # removing ques & ans header
    post = re.sub(r'<.*?>', '', post) # removing html tags

    post = re.sub(r'[^a-zA-Z0-9]'," ", post.lower()) # removing punctuation & convert to lower case
    post = re.sub('\s+', ' ', post) # removing tab or more than one consecutive space
    post = post.strip()
    return post
# for i in df_new.post.values[:10]:
#     regex(i)

# remove only digit token
def rmdigit(token):
    token = re.sub(r'^\d+$', "", token)
    return token

In [504]:
# tokenization
import nltk
import enchant

from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, words

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
dict_enchant = enchant.Dict("en_US")

def preprocessing(text):
    text = regex(text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # tokens = [word if dict_enchant.check(word) else dict_enchant.suggest(word)[0] for word in tokens]
    # tokens = [word for word in tokens if dict_enchant.check(word)]
    tokens = [word for word in tokens if len(rmdigit(word))>0]

    return tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jonat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [505]:
import sklearn
from sklearn.model_selection import train_test_split
# preparation for data modeling
X = df_new.post.values
y = df_new.overall_ans.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [506]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix

vect = CountVectorizer(tokenizer=preprocessing)
rfidf = TfidfTransformer()

vect = CountVectorizer()
tfidf = TfidfTransformer()

# words -> int
X_train = vect.fit_transform(X_train)
X_train = tfidf.fit_transform(X_train)

X_test = vect.transform(X_test)
X_test = tfidf.transform(X_test)

In [507]:
filename = ['X_train', 'X_test', 'y_train', 'y_test']
for n, name in enumerate([X_train, X_test, y_train, y_test]):
    try:
        name = name.toarray()
    except AttributeError:
        pass
    np.savez_compressed(f'{filename[n]}.npz', np.asarray(name))